In [12]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [15]:
!kaggle datasets download -d gti-upm/leapgestrecog

Dataset URL: https://www.kaggle.com/datasets/gti-upm/leapgestrecog
License(s): CC-BY-NC-SA-4.0
100% 2.13G/2.13G [00:10<00:00, 243MB/s]
100% 2.13G/2.13G [00:10<00:00, 218MB/s]


In [16]:
import zipfile
zip_ref=zipfile.ZipFile('/content/leapgestrecog.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [17]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
import numpy as np
import os
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array,ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [18]:

def gather_image_paths_and_labels(root_dir):
    file_paths = []
    labels = []

    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                file_paths.append(os.path.join(root, file))
                label = os.path.basename(os.path.dirname(root))
                labels.append(label)

    return file_paths, labels
file_paths, labels = gather_image_paths_and_labels("/content/leapgestrecog")


df = pd.DataFrame({
    'file_path': file_paths,
    'label': labels
})

df.head()


,file_path,label
0,/content/leapgestrecog/leapGestRecog/00/05_thu...,00
1,/content/leapgestrecog/leapGestRecog/00/05_thu...,00
2,/content/leapgestrecog/leapGestRecog/00/05_thu...,00
3,/content/leapgestrecog/leapGestRecog/00/05_thu...,00
4,/content/leapgestrecog/leapGestRecog/00/05_thu...,00


In [19]:
img_height, img_width = 224, 224
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_dataframe(
    df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 16000 validated image filenames belonging to 10 classes.
Found 4000 validated image filenames belonging to 10 classes.


In [20]:


num_classes = len(train_generator.class_indices)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [21]:
epochs = 1

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


500/500 [==============================] - 75s 146ms/step - loss: 0.1636 - accuracy: 0.9441 - val_loss: 16.1303 - val_accuracy: 0.0000e+00


In [22]:
validation_loss, validation_acc = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f"Validation Accuracy: {validation_acc}")


125/125 [==============================] - 13s 105ms/step - loss: 16.1303 - accuracy: 0.0000e+00
Validation Accuracy: 0.0


In [23]:


def predict_hand_gesture(img_path, model):
    img = load_img(img_path, target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction[0])

    class_indices = {v: k for k, v in train_generator.class_indices.items()}
    gesture = class_indices[predicted_class]

    return gesture


img_path = '/content/leapgestrecog/leapGestRecog/00/01_palm/frame_00_01_0001.png'
print(f"Predicted Gesture: {predict_hand_gesture(img_path, model)}")


1/1 [==============================] - 0s 67ms/step
Predicted Gesture: 08
